# Reminder workflow

<img src="images/Screenshot 2025-01-22 alle 17.50.35.png" />


# Predicting heart disease using machine learning

This notebook looks into using various Python-based machine learning and data science libraries in an attempt to build a machine learning model capable of predicting wheter or not someone has heart disease based on their medical attributes.

We're going to take the following approach:
1. Problem definition
2. Data
3. Evaluation
4. Features
5. Modelling
6. Experimentation

## 1. Problem Definition
> Dati dei parametri clinici di un paziente, possiamo prevedere se avrà o meno un attacco di cuore?

## 2. Data
A data dictionary describes the data you're dealing with.

Not all datasets come with them so this is where you may have to do your research or ask a subject matter expert (someone who knows about the data) for more.

https://archive.ics.uci.edu/dataset/45/heart+disease

## 3. Evaluation
> Se riusciamo a raggiungere una precisione (accuracy) del 95% nel prevedere se un paziente ha o meno una malattia cardiaca durante la prova di concetto, porteremo avanti il ​​progetto.

## 4. Features
**Create a data dictionary**
Possiamo crearlo da soli, oppure consultare un esperto del settore, per capire cosa significano nello specifico le varie feature e quali sono realmente utili e quali potrebbero essere eliminate.

Nel dataset, dalla documentazione da cui lo abbiamo preso, scopriamo che di 76 attributi ne è stato usato un subset di 14

<img src ="images/Screenshot 2025-01-22 alle 18.50.05.png" width=400>